## Publishers

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

# parse books json
df = pd.read_json (r'C:/Users/IliaMalfa/DataManagment1/goodreads_books_comics_graphic.json', lines=True)

# remove empty publisher name rows
df = df[~df['publisher'].isin([''])]
df = df.drop_duplicates(subset=['publisher'])
publisherDataFrame = df[['publisher','language_code']].copy()

print (df.columns.values);
print(df.shape)
df.head(20)

In [ ]:
# Generate random contact phone
contactPhoneData =np.random.randint(low=1000000000, high=9999999999,size =len(publisherDataFrame), dtype=np.int64)
publisherDataFrame['contact_phone'] = contactPhoneData
publisherDataFrame.head(10)

In [ ]:
# Generate random adress
from faker import Faker
fake = Faker()
fakeAddresses = []

for _ in range(len(publisherDataFrame)):
    fakeAddresses.append(format(fake.address()))

In [ ]:
# check that the lenght of address data created is equal to the dataframe length
len(fakeAddresses)

In [ ]:
# create address column at publisherDataFrame
publisherDataFrame['address'] = fakeAddresses
publisherDataFrame.head(10)

In [ ]:
# rename publisherDataFrame columns
publisherDataFrame.rename(columns = {'publisher' :'name'}, inplace = True)
publisherDataFrame.rename(columns = {'language_code':'country_of_headquarters'}, inplace = True)
publisherDataFrame.head(10)

In [ ]:
# WAY 2 QUICK WAY
# connect to the db and pass the data
import os
import psycopg2
import numpy as np
import psycopg2.extras as extras
from io import StringIO

param_dic = {
    "host"      : "localhost",
    "database"  : "ComicShop",
    "user"      : "postgres",
    "password"  : "mypass"
}


def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn
conn = connect(param_dic)

def execute_values(conn, df, table):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    print(table)
    print(cols)
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()
    
table = "publishers"
execute_values(conn, publisherDataFrame, table)

## Authors

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

# parse authors json
df = pd.read_json (r'C:/Users/IliaMalfa/DataManagment1/goodreads_book_authors.json', lines=True)

print (df.columns.values);
print(df.shape)
df.head(20)

In [ ]:
# create a authors dataframe with the desirable columns
authorsDataFrame = df[['author_id','name']].copy()

In [ ]:
# Guess the gender by the firstName, 
# if the first argument of name is a salutation and contains . then check the second argument
import gender_guesser.detector as gender
d = gender.Detector()
data =[]

df.name.replace(to_replace='', value=None, regex=True,inplace=True) 
for index, row in df.iterrows():
    firstName= row['name'].split()[0]
    try:
        firstName.index(".")
    except ValueError:
        print("occurred an exception")
    else:
         try:
            firstName= row['name'].split()[1]
         except:
            print("here occurred an exception")
         else:       
            firstName= row['name'].split()[1]
    gender = d.get_gender(firstName)
    print(firstName +" :"+gender)
    data.append(format(gender))

In [ ]:
# check that the lenght of gender data created is equal to the dataframe length
len(data)

In [ ]:
# create gender column at authorsDataFrame
authorsDataFrame['gender'] = data
authorsDataFrame

In [ ]:
# rename authorsDataFrame columns
authorsDataFrame.rename(columns = {'author_id':'id'}, inplace = True)
authorsDataFrame.rename(columns = {'name':'author_name'}, inplace = True)

In [ ]:
# WAY 2 QUICK WAY
# connect to the db and pass the data
import os
import psycopg2
import numpy as np
import psycopg2.extras as extras
from io import StringIO

param_dic = {
    "host"      : "localhost",
    "database"  : "ComicShop",
    "user"      : "postgres",
    "password"  : "mypass"
}


def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn
conn = connect(param_dic)

def execute_values(conn, df, table):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()
    
table = "authors"
execute_values(conn, authorsDataFrame, table)

## Books

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

# path to books json file
p = Path(r'C:/Users/IliaMalfa/DataManagment1/goodreads_books_comics_graphic.json')

# load json
books = []
for line in open(p, 'r'):
    books.append(json.loads(line))

# create dataframe with normalized authors for the author values that are nested
df = pd.json_normalize(books, record_path='authors', meta=['isbn','title','author_id', 'publisher','publication_year', 'description', 'book_id'], record_prefix='authors_',errors='ignore')

# remove empty isbn rows and drow isbn duplicates
df = df[~df['isbn'].isin([''])]
df = df.drop_duplicates(subset=['isbn'])
bookDataframe = df[['isbn','title', 'authors_author_id', 'publisher', 'publication_year', 'description', 'book_id']].copy()


bookDataframe.head(10)

In [ ]:
# add price column as a fixed point attribute with 2 decimal digits
data = np.round(np.random.uniform(5,40,len(bookDataframe)),2)
bookDataframe['price'] = data

In [ ]:
# rename publisherDataFrame columns
bookDataframe.rename(columns = {'authors_author_id' :'author_id'}, inplace = True)
bookDataframe.rename(columns = {'description':'short_description'}, inplace = True)
bookDataframe.rename(columns = {'book_id':'id'}, inplace = True)
bookDataframe.head(10)

In [ ]:
# WAY 2 QUICK WAY
# connect to the db and pass the data
import os
import psycopg2
import numpy as np
import psycopg2.extras as extras
from io import StringIO

param_dic = {
    "host"      : "localhost",
    "database"  : "ComicShop",
    "user"      : "postgres",
    "password"  : "mypass"
}


def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn
conn = connect(param_dic)

def execute_values(conn, df, table):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    print(table)
    print(cols)
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()
    
table = "books"
execute_values(conn, bookDataframe, table)

# Authors With Books And Roles

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

# path to books json file
p = Path(r'C:/Users/IliaMalfa/DataManagment1/goodreads_books_comics_graphic.json')

# load json
books = []
for line in open(p, 'r'):
    books.append(json.loads(line))

# create dataframe with normalized authors for the author values that are nested
df = pd.json_normalize(books, record_path='authors', meta=['isbn','title','author_id', 'role', 'publisher','publication_year', 'description', 'language_code', 'book_id'], record_prefix='authors_',errors='ignore')

# remove empty isbn
df = df[~df['isbn'].isin([''])]
df.head(20)


In [ ]:
# keep author_id and role to make it a new table
authorsWithRolesAndNationalityDataframe = df[['authors_author_id','isbn','authors_role', 'language_code', 'book_id']].copy()
authorsWithRolesAndNationalityDataframe.head(10)

In [ ]:
# rename authorsWithRolesAndNationalityDataframe columns
authorsWithRolesAndNationalityDataframe.rename(columns = {'authors_author_id' :'author_id'}, inplace = True)
authorsWithRolesAndNationalityDataframe.rename(columns = {'isbn':'book_isbn'}, inplace = True)
authorsWithRolesAndNationalityDataframe.rename(columns = {'authors_role':'author_role'}, inplace = True)
authorsWithRolesAndNationalityDataframe.rename(columns = {'language_code':'author_nationality'}, inplace = True)

In [ ]:
# WAY 2 QUICK WAY
# connect to the db and pass the data
import os
import psycopg2
import numpy as np
import psycopg2.extras as extras
from io import StringIO

param_dic = {
    "host"      : "localhost",
    "database"  : "ComicShop",
    "user"      : "postgres",
    "password"  : "mypass"
}


def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn
conn = connect(param_dic)

def execute_values(conn, df, table):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    print(table)
    print(cols)
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()
    
table = "bookswithauthorsandroles"
execute_values(conn, authorsWithRolesAndNationalityDataframe, table)

# Book Reviews

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

# parse authors json
df = pd.read_json (r'C:/Users/IliaMalfa/DataManagment1/goodreads_reviews_comics_graphic.json', lines=True)
print (df.columns.values);
print(df.shape)
df.head(20)

In [ ]:
# remove empty isbn
df = df[~df['book_id'].isin([''])]
df.shape

In [ ]:
# create a authors dataframe with the desirable columns
reviewsDataFrame = df[['review_id','book_id','rating','review_text', 'date_added', 'user_id']].copy()
df.shape

In [ ]:
from datetime import datetime

data= []

for index,row in df.iterrows():
    datetime_object = datetime.strptime(row['date_added'], '%a %b %d %H:%M:%S -%f %Y')
    data.append(datetime_object)

In [ ]:
print(len(data))

In [ ]:
# rename reviewsDataFrame columns
reviewsDataFrame.rename(columns = {'rating' :'score'}, inplace = True)
reviewsDataFrame.rename(columns = {'date_added':'creation_timestamp'}, inplace = True)
reviewsDataFrame.rename(columns = {'user_id':'nickName'}, inplace = True)
reviewsDataFrame.head(20)

In [ ]:
# Generate nicknames based on every user grouped by the userId
from faker import Faker
from random import randint

group_by_userId = reviewsDataFrame.groupby(["nickName"])["nickName"]
type(group_by_userId)
len(group_by_userId)
for i in group_by_userId:
    reviewsDataFrame.loc[reviewsDataFrame['nickName'] == i[0], ['nickName']] = format(fake.name().split()[0]+str(randint(10,99)))

In [ ]:
reviewsDataFrame.head(20)

In [ ]:
reviewsDataFrame['creation_timestamp'] = data
reviewsDataFrame.shape

In [ ]:
# WAY 2 QUICK WAY
# connect to the db and pass the data
import os
import psycopg2
import numpy as np
import psycopg2.extras as extras
from io import StringIO

param_dic = {
    "host"      : "localhost",
    "database"  : "ComicShop",
    "user"      : "postgres",
    "password"  : "mypass"
}


def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn
conn = connect(param_dic)

In [ ]:
def execute_values(conn, df, table):
    """
    Using psycopg2.extras.execute_values() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    print(table)
    print(cols)
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_values() done")
    cursor.close()
    
table = "reviews"
execute_values(conn, reviewsDataFrame, table)